In [2]:
import pandas as pd
from datasets import load_dataset
import json

/home/lab/.conda/envs/upstream_sdg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fp = "/new_data/wenlong_rh/wz-entigraph-data/data/dataset/raw/entigraph_raw_dataset.jsonl"
df = pd.read_json(fp, lines=True)
fp = "/new_data/wenlong_rh/wz-entigraph-data/data/dataset/subset-selected-dataset/entigraph_raw_dataset/entigraph_raw_dataset_percent_10.0_subset.jsonl"
df_10 = pd.read_json(fp, lines=True)
df_10_uid = df_10.merge(
    df[['text', 'uid']].drop_duplicates(subset=['text']), 
    on='text', 
    how='left'
)
with open("task_subset_24.json", "r") as f:
    selected_uid = json.load(f)

In [3]:
len(selected_uid)

24

In [4]:
df_10_uid_24 = df_10_uid[df_10_uid['uid'].isin(selected_uid)]

In [5]:
df_10_uid_24

,text,uid
5,"### Discussion of ""Bodyguard"" by H. L. Gold in...","Bodyguard by Gold, H. L. (Horace Leonard)"
146,"### Discussion of ""A Good Year for the Roses?""...",A Good Year for the Roses? by David Edelstein
215,"### Discussion of ""A Good Year for the Roses?""...",A Good Year for the Roses? by David Edelstein
235,"### Discussion of ""A Good Year for the Roses?""...",A Good Year for the Roses? by David Edelstein
301,"### Discussion of ""Fight Clubbed"" in relation ...",Fight Clubbed by David Plotz
...,...,...
74195,"### Discussion of ""My Father's Estate"" in rela...",My Father's Estate by Ben Stein
74196,"### Discussion of ""Fight Clubbed"" in relation ...",Fight Clubbed by David Plotz
74198,"### Discussion of ""Fight Clubbed"" in relation ...",Fight Clubbed by David Plotz
74214,"### Discussion of ""Beach Scene"" in relation to...","Beach Scene by King, Marshall"


In [6]:
df_10_uid.to_json("/new_data/wenlong_rh/wz-entigraph-data/data/dataset/subset-selected-dataset/entigraph_raw_dataset/entigraph_raw_dataset_percent_10.0_subset_uid.jsonl", orient="records", lines=True)
df_10_uid_24.to_json("/new_data/wenlong_rh/wz-entigraph-data/data/dataset/subset-selected-dataset/entigraph_raw_dataset/entigraph_raw_dataset_percent_10.0_subset_uid_first_24.jsonl", orient="records", lines=True)

In [10]:
ds = load_dataset('json', data_files="/new_data/wenlong_rh/wz-entigraph-data/data/dataset/subset-selected-dataset/entigraph_raw_dataset/entigraph_raw_dataset_percent_10.0_subset_uid_first_24.jsonl", split="train")
ds = ds.add_column("route", ['_A_']*len(ds))
ds = ds.rename_columns({"text": "document", "uid": "document_outline"})
icl = {
    'icl_document': '### Discussion of "Fight Clubbed" in relation to Mike Tyson\nThe article\'s examination of the Ultimate Fighting Championship (UFC) landscape brings notable clashes into perspective, including the hypothetical scenario where a juggernaut of traditional boxing like Mike Tyson steps into the UFC\'s unforgiving realm. Contrasted against the diversity of disciplines displayed in UFC, Tyson, a titan in the boxing ring with knockout power that is the stuff of legend, receives a provocative dismissal. The assertion states unequivocally that Tyson would scarcely last half a minute in an ultimate fighting match. Such a statement not only elevates the standing of UFC fighters as the epitome of versatile combat proficiency, but it also underscores Tyson\'s specialized yet limited skill set, which may not be as effective in the no-holds-barred environment that UFC champions.\n\n### Discussion of "Fight Clubbed" in relation to San Pedro, California\nThe piece on the violent contrasts of UFC and Hollywood\'s portrayal of fighting ends with an indication of the subculture\'s persistence, showcasing outliers like the Kage Kombat competitions taking place in San Pedro, California. This reference to San Pedro, a microcosm within the larger narrative, captures the UFC\'s retreat from mainstream visibility following heavy political and cultural resistance that relegated it to more discreet locales. San Pedro becomes an unlikely haven for the continued, albeit overshadowed, tradition of mixed martial arts, where the fundamental spirit of UFC is kept alive away from the grandiose stages and bright lights.\n\n### Discussion of Interaction between Mike Tyson and San Pedro, California in context of "Fight Clubbed"\nIn "Fight Clubbed," Mike Tyson and San Pedro, California emerge as vivid symbols within the broader narrative of UFC\'s contentious journey. Tyson, an embodiment of elite, mainstream boxing prowess, is used to starkly contrast the brutal effectiveness and diverse skill requirements of UFC fighting. Meanwhile, San Pedro embodies the UFC\'s shift from a publicly celebrated spectacle to a niche pursuit echoing in the underground echoes of fisticuff folklore. Tyson\'s hypothetical inadequacy within the UFC octagon implicitly suggests a broader theme of adaptability and evolution in combat sports, a theme mirrored by San Pedro\'s role as an inconspicuous sanctuary for a suppressed yet persistent fighting community. While the article does not describe any direct interaction between Tyson and San Pedro, the juxtaposition of these two entities speaks volumes about the changing face of American martial arts and the cultural tug-of-war between visibility and existence in the penumbra of public consciousness.', 
    "icl_query_1": "How does 'Fight Clubbed' characterize Mike Tyson’s abilities in the context of the UFC?",
    "icl_response_1": "The article uses Mike Tyson, a legendary boxer known for his knockout power, to highlight the limitations of traditional boxing in the context of UFC. It argues that Tyson would barely last thirty seconds in a UFC fight, emphasizing the superior versatility and combat proficiency required in mixed martial arts compared to specialized boxing skills.",
    "icl_query_2": "What role does San Pedro, California play in the article’s discussion of the UFC’s cultural trajectory?",
    "icl_response_2": "San Pedro, California is presented as a symbolic location that captures the UFC’s retreat from mainstream popularity due to political and cultural pushback. Through underground events like Kage Kombat, it illustrates how the spirit of mixed martial arts persists in obscure, localized venues even after losing broader public visibility.",
    "icl_query_3": "How are Mike Tyson and San Pedro, California used together to reflect broader themes in 'Fight Clubbed'?",
    "icl_response_3": "Tyson and San Pedro serve as contrasting symbols in the article: Tyson represents the traditional, mainstream combat sports world, while San Pedro embodies the UFC’s marginalized yet resilient underground scene. Their juxtaposition conveys themes of adaptability in combat sports and highlights the cultural struggle between visibility and survival in evolving martial arts landscapes.",
}
ds = ds.map(lambda x: icl)
ds.to_json("/new_data/knowledge_rh/quality/base_datasets/entigraph_raw_dataset_percent_10.0_subset_uid_first_24.jsonl", orient="records", lines=True)

Creating json from Arrow format: 100%|██████████| 7/7 [00:00<00:00, 27.90ba/s]


51582941

```shell
# Get dataset size and save into variable
dataset_size=$(wc -l /new_data/knowledge_rh/quality/base_datasets/quality_seed_example_chunked.jsonl | awk '{print $1}')
number_of_processes=8
port=8000
for i in {0..7}; do
    dataset_start_index=$((i * dataset_size / number_of_processes))
    dataset_end_index=$((dataset_start_index + dataset_size / number_of_processes))
    python scripts/generate.py --ds_path  /new_data/knowledge_rh/quality/base_datasets/entigraph_knowledge_icl_10_percent.jsonl \
        --bs 2 --num_workers 30 \
        --save_path /new_data/knowledge_rh/quality/entigraph_knowledge1.0_phi4/gen.jsonl \
        --flow /home/lab/abhi/SDG-Research-Upstream/src/instructlab/sdg/flows/generation/knowledge/synth_knowledge_phi4.yaml \
        --endpoint http://localhost:$port/v1 \
        --checkpoint_dir /new_data/knowledge_rh/quality/entigraph_knowledge1.0_phi4/data_checkpoints \
        --save_freq 1000 \
        --dataset_start_index $dataset_start_index \
        --dataset_end_index $dataset_end_index > run_sdg_$i.log 2>&1 &
    echo "Starting process $i with dataset from $dataset_start_index to $dataset_end_index on port $port"
    port=$((port + 1))
done
```

In [1]:
enti_ds = load_dataset('json', data_dir="/new_data/knowledge_rh/quality/entigraph_knowledge1.0_phi4_first_24_n_5/gen/", split="train")
enti_ds = enti_ds.filter(lambda x: x['score'] == '2' and x['judgment'] == 'YES')
enti_ds.to_json("/new_data/knowledge_rh/quality/entigraph_knowledge1.0_phi4_first_26/gen.jsonl", orient="records", lines=True)

NameError: name 'load_dataset' is not defined

In [36]:
from instructlab.sdg.flow import  Flow
from instructlab.sdg.logger_config import setup_logger
from instructlab.sdg.pipeline import Pipeline
from instructlab.sdg.sdg import SDG
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )
flow_cfg = Flow(client).get_flow_from_file("/home/lab/abhi/SDG-Research-Upstream/src/instructlab/sdg/flows/generation/knowledge/knowledge_filtering_phi4.yaml")
sdg = SDG(
        [Pipeline(flow_cfg)],
        num_workers=1,
        batch_size=1,
        save_freq=1000,
    )
    

In [ ]:
generated_data = sdg.generate(enti_ds.select(range(20)), checkpoint_dir="tmp")